<a href="https://colab.research.google.com/github/Bora-Ulu/Inflation-Technique/blob/main/Inflation_Technique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from itertools import *
import time
from scipy.sparse import coo_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import csr_matrix
from cvxopt import matrix, solvers, sparse, spmatrix
from numba import njit

"""
@njit
def PrepForIntToArray32(ar):# Takes a huge column vector as input
                     # Returns a matrix where the rows are the base-4 binary expansions of each element of the input array
    #return transpose(atleast_2d(ravel(ar.astype(uint32)))).view(uint8)
    return np.reshape(ar.astype(np.uint32).view(np.uint8),(-1,4))


def IntToArray32(ar):
    return np.unpackbits(PrepForIntToArray32(ar),axis=1, bitorder='little').view(np.bool_)


def ArrayToInt32(bitarray):
    return np.packbits(bitarray, bitorder='little').view(np.uint32)

@njit
def ExtractBitsFromBitArray(bitarray,bitstoextract): #bitstoextract must be numpy array
    tempbitarray=np.zeros_like(bitarray)
    tempbitarray[:,:len(bitstoextract)]=bitarray[:,np.subtract(np.amax(bitstoextract),bitstoextract)]
    return tempbitarray

def ExtractBitsFromInts(intarray,bitstoextract): #bitstoextract need not be a numpy array

    return ArrayToInt32(ExtractBitsFromBitArray(IntToArray32(intarray),np.array(bitstoextract,np.uint16)))
""";

@njit
def PositionIndex(arraywithduplicates):
    arraycopy=np.zeros_like(arraywithduplicates)
    u=np.unique(arraywithduplicates)
    arraycopy[u]=np.arange(len(u))
    return arraycopy[arraywithduplicates]

In [2]:
card=4
num_var=12
X=np.array([[0,1,8,10],[2,3,9,11]])
Y=np.array([[0,2,4,5],[1,3,6,7]])
Z=np.array([[4,6,8,9],[5,7,10,11]])
inflation_order=2

anc_con=np.array([X,Y,Z])


In [3]:
def swapcols(swap,l):  
    pregen=np.arange(l)
    #smartswap=np.reshape(swap,(2,-1))
    pregen[swap] = pregen[np.array([swap[1],swap[0]])]
    return pregen

    
def GroupGeneratorsFromSwaps(anc_con):
    #return np.apply_along_axis(swapcols,-1,np.reshape(anc_con,(len(anc_con),-1)))
    #return np.apply_along_axis(swapcols,-1,anc_con)
    l=np.amax(anc_con)+1
    return np.array(list(swapcols(swap,l) for swap in anc_con))
    #    yield swapcols(swap)
group_gens=GroupGeneratorsFromSwaps(anc_con)


def subsets(s):
    return tuple(np.array(list(combinations(list(s), cardinality))) for cardinality in range(2,len(s)+1))

#@njit
import functools
def reindex(repeatedindexing):
    #final=repeatedindexing[0]
    #for i in range(len(repeatedindexing[1:])):
    #    final=final[repeatedindexing[i]]
    return functools.reduce(lambda x, y: x[y], repeatedindexing)

#print(np.array(list(reindex(elem) for elem in subsets(group_gens)[0])))

def GroupElementsFromGenerators(group_gens):
    return np.vstack((np.arange(np.shape(group_gens)[-1]),group_gens,np.vstack(list(np.array(list(reindex(elem) for elem in combos)) for combos in subsets(group_gens)))))

group_elem=GroupElementsFromGenerators(group_gens)

group_elem

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [ 2,  3,  0,  1,  4,  5,  6,  7,  9,  8, 11, 10],
       [ 1,  0,  3,  2,  6,  7,  4,  5,  8,  9, 10, 11],
       [ 0,  1,  2,  3,  5,  4,  7,  6, 10, 11,  8,  9],
       [ 3,  2,  1,  0,  6,  7,  4,  5,  9,  8, 11, 10],
       [ 2,  3,  0,  1,  5,  4,  7,  6, 11, 10,  9,  8],
       [ 1,  0,  3,  2,  7,  6,  5,  4, 10, 11,  8,  9],
       [ 3,  2,  1,  0,  7,  6,  5,  4, 11, 10,  9,  8]])

In [4]:
def GenerateAMatrix(card, num_var, anc_con):

    AMatrix=np.vstack(tuple(
        np.ravel(np.transpose(
            np.reshape(np.arange(0,card**num_var,1,np.uint32),tuple(np.multiply(card,np.ones(num_var,np.uint8)))),tuple(elem)
        )) for elem in GroupElementsFromGenerators(GroupGeneratorsFromSwaps(anc_con))
    ))
    print(AMatrix.shape)
    AMatrix=AMatrix[:,np.unique(np.amin(AMatrix,axis=0))]
    print(AMatrix.shape)
    return AMatrix

In [5]:
def MergeMonomials(bvector,encoding):
    return np.ravel(coo_matrix((bvector, (np.zeros(len(bvector),np.uint8), encoding)),(1, np.amax(encoding)+1)).toarray())

In [6]:
def GenerateEncodingMonomialToRow(original_cardinality_product,inflation_order):
    return PositionIndex(np.amin(
    np.vstack(tuple(
        np.ravel(np.transpose(np.reshape(np.arange(original_cardinality_product**inflation_order),tuple(np.multiply(original_cardinality_product,np.ones(inflation_order,np.uint8)))),elem
                              )) for elem in list(permutations(np.arange(inflation_order)))
)),axis=0))
    
original_cardinality_product=4**3
inflation_order=2
EncodingMonomialToRow=GenerateEncodingMonomialToRow(original_cardinality_product,inflation_order)
np.amax(EncodingMonomialToRow)

2079

In [7]:
def GenerateEncodingColumnToMonomial(card,num_var,ai_naturalorder):
    ColumnIntegers=np.reshape(np.arange(0,card**num_var,1,np.uint32),tuple(np.multiply(card,np.ones(num_var,np.uint8))))
    ai_everythingelse=np.arange(num_var)[~np.array([(i in ai_naturalorder) for i in np.arange(num_var)])]
    ColumnIntegersRearranged=np.transpose(ColumnIntegers,tuple(np.hstack((ai_everythingelse,ai_naturalorder))));
    ColumnIntegersRearranged=np.reshape(ColumnIntegersRearranged,(-1,card**len(ai_naturalorder)))
    EncodingColumnToMonomial=np.zeros(card**num_var,np.uint32)
    EncodingColumnToMonomial[ColumnIntegersRearranged]=np.arange(card**len(ai_naturalorder))
    #for i in np.arange(card**len(ai_naturalorder)):   
    #    EncodingColumnToMonomial[ColumnIntegersRearranged[:,i]]=i
    return EncodingColumnToMonomial

ai_naturalorder=np.array([1,5,9,4,8,12])-1
EncodingColumnToMonomial=GenerateEncodingColumnToMonomial(card,num_var,ai_naturalorder)
np.amax(EncodingColumnToMonomial)

4095

In [8]:
def MergeMonomials(bvector,encoding):
    return np.ravel(coo_matrix((bvector, (np.zeros(len(bvector),np.uint8), encoding)),(1, np.amax(encoding)+1)).toarray())

In [9]:
def EncodeA_FindB(card, num_var, anc_con, ai_naturalorder, Data, inflation_order):
    
    EncodingMonomialToRow=GenerateEncodingMonomialToRow(len(Data),inflation_order)
    
    EncodingColumnToMonomial=GenerateEncodingColumnToMonomial(card,num_var,ai_naturalorder)
    #EncodingColumnToMonomial=ArrayToInt32(ExtractBitsFromBitArray(ColumnIntegersAsBits,np.array(ai_naturalbitorder,np.uint16)))
    
    EncodedA=EncodingMonomialToRow[EncodingColumnToMonomial][GenerateAMatrix(card, num_var, anc_con)]
    
    preb=np.array(Data)
    b=preb
    for i in range(1,inflation_order):
        b=np.kron(preb,b)

    b=MergeMonomials(b,EncodingMonomialToRow)

    return EncodedA,b

In [10]:
def FormCVXOPTArrayFromOnesPositions(OnesPositions):
    columncount=OnesPositions.shape[-1]
    columnspec=np.ravel(np.broadcast_to(np.arange(columncount), (len(OnesPositions), columncount)))
    return spmatrix(np.ones(OnesPositions.size), np.ravel(OnesPositions).tolist(), columnspec.tolist(),(np.amax(OnesPositions)+1, columncount))

In [11]:
Data=[0.12199995751046305, 0.0022969343799089472, 0.001748319476328954, 3.999015242496535e-05, 0.028907881434196828, 0.0005736087488455967, 0.0003924033706699725, 1.1247230369521505e-05, 0.0030142577390317635, 0.09234476010282468, 4.373922921480586e-05, 0.0014533921021948346, 0.0007798079722868244, 0.024091567451515063, 1.1247230369521505e-05, 0.0003849052170902915, 0.020774884184769502, 0.000396152447459813, 0.0003049249122403608, 4.998769053120669e-06, 0.10820335492385, 0.0020794879260981982, 0.0015546171755205281, 2.4993845265603346e-05, 0.0006260958239033638, 0.020273757587194154, 7.498153579681003e-06, 0.0003374169110856452, 0.0028942872817568676, 0.08976414557915113, 2.624353752888351e-05, 0.0012984302615480939, 0.002370666223442477, 4.7488306004646356e-05, 0.0999928767540993, 0.001957018084296742, 0.0006198473625869629, 8.747845842961171e-06, 0.02636975644747481, 0.0005198719815245496, 1.4996307159362007e-05, 0.000403650601039494, 0.0005498645958432735, 0.017359475229224805, 7.123245900696953e-05, 0.002346922070440154, 0.0033754188031197316, 0.10295964618712641, 0.00038740460161685187, 7.498153579681003e-06, 0.01608353942841575, 0.000306174604503641, 0.0021319750011559654, 4.248953695152569e-05, 0.09107007399427891, 0.001860791780024169, 5.998522863744803e-05, 0.0018395470115484063, 0.002570616985567304, 0.0766411271224461, 1.874538394920251e-05, 0.00048238121362614454, 0.0006410921310627258, 0.020223769896662948]
inflation_order=2;
#ai_naturalbitorder=[0,1,8,9,16,17,6,7,14,15,22,23]
#ai_naturalbitorder=UpgradeToBits(ai_naturalorder,2)
start = time.time()
EncodedA,b=EncodeA_FindB(card, num_var, anc_con, ai_naturalorder,Data,inflation_order)
MCVXOPT=FormCVXOPTArrayFromOnesPositions(EncodedA).T #This far the code is generalized
print('It took', time.time()-start, 'seconds.')
print(MCVXOPT.size)

(8, 16777216)
(8, 2123776)
It took 3.5379981994628906 seconds.
(2123776, 2080)


In [ ]:
rowcount=MCVXOPT.size[0];
colcount=MCVXOPT.size[1];
CVXOPTb=matrix(np.atleast_2d(b).T)
CVXOPTh=matrix(np.zeros((rowcount,1)))
CVXOPTA=matrix(np.ones((1,colcount)))
solvers.lp(CVXOPTb,-MCVXOPT,CVXOPTh,CVXOPTA,matrix(np.ones((1,1))),solver='mosek')

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2123777         
  Cones                  : 0               
  Scalar variables       : 2080            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.25            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 1.33    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization 

In [ ]:
#A1 A2 A3 A4 B1 B2 B3 B4 C1 C2 C3 C4
#0. 1. 2. 3. 4. 5. 6. 7. 8. 9. 10 11

#A1. A2. A3. A4. B1. B2.   B3.   B4.   C1.   C2.   C3.   C4
#0,1 2,3 4,5 6,7 8,9 10,11 12,13 14,15 16,17 18,19 20,21 22,23

#A1 A4 B1 B4 C1 C4
#0. 1. 2. 3. 4. 5